In [1]:
import os
import warnings
warnings.filterwarnings("ignore")
import pickle
import imageio

import pygmt
import numpy as np
import pandas as pd
import xarray as xr
import xesmf as xe
from numpy import deg2rad, sin, cos, meshgrid, gradient

from fcts import *

from pygmt.datasets import load_earth_relief
grid = load_earth_relief(resolution="06m", registration="gridline")

In [2]:
rotation='../paleo_reef/paleomap/PALEOMAP_PlateModel.rot'
plate='../paleo_reef/paleomap/PALEOMAP_PlatePolygons.gpml'

Rearth = 6371.*1000.0

In [3]:
def earth_radius(lat):
    a = 6378137
    b = 6356752.3142
    e2 = 1 - (b**2/a**2)
    lat = deg2rad(lat)
    lat_gc = np.arctan( (1-e2)*np.tan(lat) )
    r = (
        (a * (1 - e2)**0.5) 
         / (1 - (e2 * np.cos(lat_gc)**2))**0.5 
        )
    return r

def area_grid(lat, lon):
    xlon, ylat = meshgrid(lon, lat)
    R = earth_radius(ylat)
    dlat = deg2rad(gradient(ylat, axis=0))
    dlon = deg2rad(gradient(xlon, axis=1))
    dy = dlat * R
    dx = dlon * R * cos(deg2rad(ylat))
    area = dy * dx
    xda = xr.DataArray(
        area,
        dims=["latitude", "longitude"],
        coords={"latitude": lat, "longitude": lon},
        attrs={
            "long_name": "area_per_pixel",
            "description": "area per pixel",
            "units": "m^2",
        },
    )
    return xda

wmean = xr.open_dataset('../paleo_reef/sims/wmean_foster/0Ma.nc') #.fillna(0)
da_area = area_grid(wmean['latitude'], wmean['longitude'])

glon = da_area.longitude.values
glat = da_area.latitude.values
glons, glats = np.meshgrid(glon, glat)

env = xr.open_dataset('../paleo_reef/env_var/enviVar0Ma_res1_foster.nc')
regrid = xe.Regridder(env, wmean, 'bilinear', periodic=True, weights='data/regrid_elev.nc')

In [4]:
kiessling03 = pd.read_csv('../paleo_reef/data/kiessling03.csv')
kiessling03

,period,meanArea,accRate,start,end
0,Permian,6200000.0,140,299,251
1,Triassic,7000000.0,230,251,199
2,Jurassic,8900000.0,167,199,145
3,Cretaceous,11000000.0,186,145,66
4,Paleogene,12000000.0,140,66,23
5,Neogene,9700000.0,177,23,0


In [5]:
def reconstructTime(tstart, tsim, depthlimit=False):

    vtime = -np.arange(-tstart,0,dt)
    env = xr.open_dataset(elevshare+str(tsim)+'Ma_res1_foster.nc')
    
    # Get the suitable habitats for a specific time
    ds = xr.open_dataset(hydroshare+str(tstart)+'Ma.nc').copy()
    regrid = xe.Regridder(env, ds, 'bilinear', periodic=True, weights='data/regrid_elev.nc')

    fb = regrid(env.bathy).where(ds.wmean>0)
    depth = fb.where(fb<0,-1).where(fb>-1400).where(ds.wmean>0)
    meandepth = -depth.mean().values+0.
    depth = depth.values.ravel().copy()
    dataMean = ds['wmean'].where(fb>-1400).values.ravel().copy()
    dataArea = da_area.values.ravel().copy()
    
    # Remove nan points and points with limited <1% suitability probability
    mask = np.where(np.logical_and(~np.isnan(dataMean),dataMean>5))
    wmean_val = dataMean[mask]
    area_val = dataArea[mask]
    depth_val = -depth[mask]
    
    lonlat = np.empty((len(wmean_val), 2))
    lonlat[:, 0] = glons.ravel()[mask]
    lonlat[:, 1] = glats.ravel()[mask]
    
    # Take the accumulation for the considered time slice
    for k in range(len(kiessling03)):
        if tstart < kiessling03['start'].iloc[k] and tstart >= kiessling03['end'].iloc[k]:
            carbrate = kiessling03['accRate'].iloc[k]
    
    # Calculate the total area covered by carbonate km2
    area_tot = ((wmean_val*area_val).sum()/1000)/1.e6
    
    # Calculate the volume of carbonate for 5 Myr in km3 for the time slice
    # and limit the volume based on shelf depth
    if depthlimit:
        th_val = np.minimum(carbrate*wmean_val*5/1000,depth_val)
    else:
        th_val = carbrate*wmean_val*5/1000
    vol_val = th_val*area_val*1.e-9 

    mvwmean_val = wmean_val.copy()
    mvarea_val = area_val.copy()
    mvvol_val = vol_val.copy()
    mvth_val = th_val.copy()
    mvlonlat = lonlat.copy()
    meshXYZ = polarToCartesian(Rearth, lonlat[:, 0], lonlat[:, 1])
    
    destroyWmean = []
    destroyCarbTh = []
    destroyCarbVol = []
    destroyArea = []
    destroy = []
                                                              
    for k in range(len(vtime)):
        sXYZ = polarToCartesian(Rearth, mvlonlat[:, 0], mvlonlat[:, 1])
        plateIds, vtree = getPlateIDs(vtime[k], mvlonlat, velshare, tree=None)
        if k > 0:
            notjumpsIDs = np.where(np.equal(pIDs, plateIds))[0]
            jumpsIDs = np.where(np.not_equal(pIDs, plateIds))[0]
            pIDs = plateIds.copy()[notjumpsIDs].copy()
            destroy.append(len(jumpsIDs))
            destroyArea.append(mvarea_val[jumpsIDs].sum()*1.e-6)
            destroyWmean.append((mvarea_val[jumpsIDs]*mvwmean_val[jumpsIDs]).sum()*1.e-6/1000)
            destroyCarbTh.append((mvth_val[jumpsIDs]).sum())
            destroyCarbVol.append((mvvol_val[jumpsIDs]).sum())
            
            plateIds = plateIds[notjumpsIDs].copy()
            sXYZ = sXYZ[notjumpsIDs,:].copy()
            mvwmean_val = mvwmean_val[notjumpsIDs].copy()
            mvarea_val = mvarea_val[notjumpsIDs].copy()
            mvth_val = mvth_val[notjumpsIDs].copy()
            mvvol_val = mvvol_val[notjumpsIDs].copy()
            mvlonlat = np.empty((len(mvwmean_val),2))
        else:
            pIDs = plateIds.copy()
            destroy.append(0)
            destroyCarbTh.append(0)
            destroyCarbVol.append(0)
            destroyArea.append(0)
            destroyWmean.append(0)
        rotations = getRotations(vtime[k], dt, plateIds, rotation, plate)
        movXYZ = movePlates(sXYZ, plateIds, rotations)
        mvlonlat[:, 0] , mvlonlat[:, 1]  = cartesianToPolarCoords(movXYZ)

    fdata = {
        'time': -vtime,
        'wmean': destroyWmean,
        'area': destroyArea,
        'vol': destroyCarbVol,
        'thick': destroyCarbTh,
        'nb': destroy,
    }
    destroyDf = pd.DataFrame(fdata)
    
    rdata = {
        'lon': mvlonlat[:,0],
        'lat': mvlonlat[:,1],
        'wmean': mvwmean_val,
        'vol': mvvol_val,
        'thick': mvth_val,
        'area': mvarea_val,
    }
    remainDf = pd.DataFrame(rdata)

    preservedArea = [area_tot.sum()-np.cumsum(np.asarray(destroyWmean))[-1],area_tot.sum()]
    preservedVol = [vol_val.sum()-np.cumsum(np.asarray(destroyCarbVol))[-1],vol_val.sum()]
    preservedTh = [th_val.sum()-np.cumsum(np.asarray(destroyCarbTh))[-1],th_val.sum()]
    ds.close()
    env.close()
    
    return destroyDf, remainDf, preservedArea, preservedVol, preservedTh, meandepth

In [6]:
def reconstruct0Ma(depthlimit=False):
    env = xr.open_dataset(elevshare+'0Ma_res1_foster.nc')
    
    ds = xr.open_dataset(hydroshare+'0Ma.nc').copy()
    regrid = xe.Regridder(env, ds, 'bilinear', periodic=True, weights='data/regrid_elev.nc')

    fb = regrid(env.bathy).where(ds.wmean>0)
    depth = fb.where(fb<0,-1).where(fb>-1400).where(ds.wmean>0)
    meandepth = -depth.mean().values+0.
    depth = depth.values.ravel().copy()
    dataMean = ds['wmean'].where(fb>-1400).values.ravel().copy()
    dataArea = da_area.values.ravel().copy()
    
    # Remove nan points and points with limited <1% suitability probability
    mask = np.where(np.logical_and(~np.isnan(dataMean),dataMean>5))
    wmean_val = dataMean[mask]
    area_val = dataArea[mask]
    depth_val = -depth[mask]
    lonlat = np.empty((len(wmean_val), 2))
    lonlat[:, 0] = glons.ravel()[mask]
    lonlat[:, 1] = glats.ravel()[mask]
    
    # Take the accumulation for the considered time slice
    for k in range(len(kiessling03)):
        if 0 < kiessling03['start'].iloc[k] and 0 >= kiessling03['end'].iloc[k]:
            carbrate = kiessling03['accRate'].iloc[k]

    # Calculate the total area covered by carbonate km2
    area_tot = ((wmean_val*area_val).sum()/1000)/1.e6
    
    # Calculate the volume of carbonate for 5 Myr in km3 for the time slice
    # and limit the volume based on shelf depth
    if depthlimit:
        th_val = np.minimum(carbrate*wmean_val*5/1000,depth_val)
    else:
        th_val = carbrate*wmean_val*5/1000
    vol_val = th_val*area_val*1.e-9 

    fdata = {
        'time': [0],
        'wmean': [0],
        'area': [0],
        'vol': [0],
        'thick': [0],
        'nb': [0],
    }
    destroyDf = pd.DataFrame(fdata)
    
    rdata = {
        'lon': lonlat[:,0],
        'lat': lonlat[:,1],
        'wmean': wmean_val,
        'vol': vol_val,
        'thick': th_val,
        'area': area_val,
    }
    remainDf = pd.DataFrame(rdata)
    
    
    preservedArea = [area_tot.sum(),area_tot.sum()]
    preservedVol = [vol_val.sum(),vol_val.sum()]
    preservedTh = [th_val.sum(),th_val.sum()]
    ds.close()
    env.close()
    
    return destroyDf, remainDf, preservedArea, preservedVol, preservedTh, meandepth

In [7]:
dt = 5
ntimes = -np.arange(-265,0,dt)

curve = 'foster'
# curve = 'smooth'
hydroshare = '../paleo_reef/sims/wmean_'+curve+'/'
elevshare = '../paleo_reef/env_var/enviVar'
edshare = '../paleo_reef/sims/erorate/'
velshare = '../paleo_reef/paleomap/pID/pID'

combt = pd.read_csv('../paleo_reef/data/combine_time.csv')
simt = combt['clim'].values[55:]

In [8]:
destroyDf = [] 
remainDf = [] 
presArea = []
presVol = []
presTh = []
meandepth = []
depthlimit = False

for t in range(len(ntimes)):
    print('Reconstruct time ',int(ntimes[t]),' Ma')
    detr, rem, prs_area, prs_vol, prs_th, md = reconstructTime(int(ntimes[t]),int(simt[t]),depthlimit)
    destroyDf.append(detr)
    remainDf.append(rem)
    presArea.append(prs_area)
    presVol.append(prs_vol)
    presTh.append(prs_th)
    meandepth.append(md)
    
print('Reconstruct time  0  Ma')
detr, rem, prs_area, prs_vol, prs_th,md = reconstruct0Ma(depthlimit)
destroyDf.append(detr)
remainDf.append(rem)
presArea.append(prs_area)
presVol.append(prs_vol)
presTh.append(prs_th)
meandepth.append(md)

# Save data on disk
stf = 'pickle/nolimit_'
if depthlimit:
    stf = 'pickle/limit_'
    
with open(stf+'destroy_'+curve+'.pkl', 'wb') as f:
    pickle.dump(destroyDf, f)
    
with open(stf+'remain_'+curve+'.pkl', 'wb') as f:
    pickle.dump(remainDf, f)
    
with open(stf+'area_'+curve+'.pkl', 'wb') as f:
    pickle.dump(presArea, f)

with open(stf+'vol_'+curve+'.pkl', 'wb') as f:
    pickle.dump(presVol, f)
    
with open(stf+'th_'+curve+'.pkl', 'wb') as f:
    pickle.dump(presTh, f)
    
with open(stf+'meandepth_'+curve+'.pkl', 'wb') as f:
    pickle.dump(meandepth, f)

Reconstruct time  265  Ma
Reconstruct time  260  Ma
Reconstruct time  255  Ma
Reconstruct time  250  Ma
Reconstruct time  245  Ma
Reconstruct time  240  Ma
Reconstruct time  235  Ma
Reconstruct time  230  Ma
Reconstruct time  225  Ma
Reconstruct time  220  Ma
Reconstruct time  215  Ma
Reconstruct time  210  Ma
Reconstruct time  205  Ma
Reconstruct time  200  Ma
Reconstruct time  195  Ma
Reconstruct time  190  Ma
Reconstruct time  185  Ma
Reconstruct time  180  Ma
Reconstruct time  175  Ma
Reconstruct time  170  Ma
Reconstruct time  165  Ma
Reconstruct time  160  Ma
Reconstruct time  155  Ma
Reconstruct time  150  Ma
Reconstruct time  145  Ma
Reconstruct time  140  Ma
Reconstruct time  135  Ma
Reconstruct time  130  Ma
Reconstruct time  125  Ma
Reconstruct time  120  Ma
Reconstruct time  115  Ma
Reconstruct time  110  Ma
Reconstruct time  105  Ma
Reconstruct time  100  Ma
Reconstruct time  95  Ma
Reconstruct time  90  Ma
Reconstruct time  85  Ma
Reconstruct time  80  Ma
Reconstruct time

In [9]:
dt = 5
ntimes = -np.arange(-265,0,dt)

curve = 'smooth'
hydroshare = '../paleo_reef/sims/wmean_'+curve+'/'
elevshare = '../paleo_reef/env_var/enviVar'
edshare = '../paleo_reef/sims/erorate/'
velshare = '../paleo_reef/paleomap/pID/pID'

combt = pd.read_csv('../paleo_reef/data/combine_time.csv')
simt = combt['clim'].values[55:]

In [10]:
destroyDf = [] 
remainDf = [] 
presArea = []
presVol = []
presTh = []
meandepth = []
depthlimit = False

for t in range(len(ntimes)):
    print('Reconstruct time ',int(ntimes[t]),' Ma')
    detr, rem, prs_area, prs_vol, prs_th, md = reconstructTime(int(ntimes[t]),int(simt[t]),depthlimit)
    destroyDf.append(detr)
    remainDf.append(rem)
    presArea.append(prs_area)
    presVol.append(prs_vol)
    presTh.append(prs_th)
    meandepth.append(md)
    
print('Reconstruct time  0  Ma')
detr, rem, prs_area, prs_vol, prs_th,md = reconstruct0Ma(depthlimit)
destroyDf.append(detr)
remainDf.append(rem)
presArea.append(prs_area)
presVol.append(prs_vol)
presTh.append(prs_th)
meandepth.append(md)

# Save data on disk
stf = 'pickle/nolimit_'
if depthlimit:
    stf = 'pickle/limit_'
    
with open(stf+'destroy_'+curve+'.pkl', 'wb') as f:
    pickle.dump(destroyDf, f)
    
with open(stf+'remain_'+curve+'.pkl', 'wb') as f:
    pickle.dump(remainDf, f)
    
with open(stf+'area_'+curve+'.pkl', 'wb') as f:
    pickle.dump(presArea, f)

with open(stf+'vol_'+curve+'.pkl', 'wb') as f:
    pickle.dump(presVol, f)
    
with open(stf+'th_'+curve+'.pkl', 'wb') as f:
    pickle.dump(presTh, f)
    
with open(stf+'meandepth_'+curve+'.pkl', 'wb') as f:
    pickle.dump(meandepth, f)

Reconstruct time  265  Ma
Reconstruct time  260  Ma
Reconstruct time  255  Ma
Reconstruct time  250  Ma
Reconstruct time  245  Ma
Reconstruct time  240  Ma
Reconstruct time  235  Ma
Reconstruct time  230  Ma
Reconstruct time  225  Ma
Reconstruct time  220  Ma
Reconstruct time  215  Ma
Reconstruct time  210  Ma
Reconstruct time  205  Ma
Reconstruct time  200  Ma
Reconstruct time  195  Ma
Reconstruct time  190  Ma
Reconstruct time  185  Ma
Reconstruct time  180  Ma
Reconstruct time  175  Ma
Reconstruct time  170  Ma
Reconstruct time  165  Ma
Reconstruct time  160  Ma
Reconstruct time  155  Ma
Reconstruct time  150  Ma
Reconstruct time  145  Ma
Reconstruct time  140  Ma
Reconstruct time  135  Ma
Reconstruct time  130  Ma
Reconstruct time  125  Ma
Reconstruct time  120  Ma
Reconstruct time  115  Ma
Reconstruct time  110  Ma
Reconstruct time  105  Ma
Reconstruct time  100  Ma
Reconstruct time  95  Ma
Reconstruct time  90  Ma
Reconstruct time  85  Ma
Reconstruct time  80  Ma
Reconstruct time